In [1]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 2.8MB/s 


In [2]:
!pip install -U pywsd

     |████████████████████████████████| 26.8MB 149kB/s 
     |████████████████████████████████| 31.6MB 111kB/s 
  Created wheel for pywsd: filename=pywsd-1.2.4-cp36-none-any.whl size=26940455 sha256=6846b4b9b33ff385a024a12e1d5da2596a2bdfbe9b9ae372ce7478332b70a692
  Stored in directory: /root/.cache/pip/wheels/71/4d/d2/405b948047f7f3851f16ab9d893ce7c1a3010182900884536b
  Created wheel for wn: filename=wn-0.0.23-cp36-none-any.whl size=31792944 sha256=8ecd5b3ec033d8f63dcd7b7e2a6c2f180ec6ab62379f68cb86fbc19177a869a3
  Stored in directory: /root/.cache/pip/wheels/56/e3/c4/886021dbf4d758dc3cb9ddaa47d7d6fc895240d83f010e6305
Successfully built pywsd wn


In [4]:
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score

import nltk 
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize 

from gensim.utils import simple_preprocess
import gensim
import gensim.downloader as api

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from sklearn.metrics import  accuracy_score, precision_score, recall_score, f1_score

from pywsd import disambiguate, simple_lesk

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Warming up PyWSD (takes ~10 secs)... 

LookupError: ignored

In [ ]:
#UNIVERSAL SENTENCE ENCODER

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

In [ ]:
def load_data():
  file_path = '/content/drive/My Drive/nlp_dataset/p2_train.csv'
  df = pd.read_csv(file_path)
  file_path_test = '/content/drive/My Drive/nlp_dataset/p2_test.csv'
  df_test = pd.read_csv(file_path_test)
  return (df,df_test)

In [ ]:
def clean_data(df, df_test):
  columns = ['precedent', 'question', 'subsequent', 'response', 'type']
  df = df.iloc[:,5:]
  df_test = df_test.iloc[:,5:]
  df = df.fillna('')
  df = df[columns].replace({'&gt;': '','<&quot;>': ''}, regex=True)
  df_test = df_test.fillna('')
  df_test = df_test[columns].replace({'&gt;': '','<&quot;>': ''}, regex=True)
  return (df, df_test)

In [ ]:
def sentiment_analyzer_scores(sentence):
    sentiment_analyzer = SentimentIntensityAnalyzer()
    score = sentiment_analyzer.polarity_scores(sentence)
    return score

In [ ]:
def create_train_test_set(df, df_test, features='baseline'):


  columns = ['precedent', 'question', 'subsequent', 'response', 'type']
  target = {'answered':0, 'attacked':1, 'irrelevant':2, 'agreed':3}
  sentiments_scale = {'neu':1, 'neg':-2, 'pos':2, 'compound':0}
  feature_columns = {0:'precedent', 1:'question', 2:'subsequent', 3:'response'}
    


  X_train, y_train = df.iloc[:,:-1], df['type']
  y_train = y_train.map(target, na_action='ignore') 
  X_test, y_test = df_test.iloc[:,:-1], df_test['type']
  y_test = y_test.map(target, na_action='ignore')

  if features == 'baseline':
    #creating training vectors
    X_train_vectors = [[] for _ in range(len(X_train.columns))]  
    for i in range(len(X_train.columns)): 
      for sentence in X_train[feature_columns[i]]:
        wordsList = nltk.word_tokenize(sentence) 
        text_tagged = nltk.pos_tag(wordsList) 
        new_text = []
        for word in text_tagged:
          new_text.append(word[0] + "/" + word[1])
        new_text = ' '.join(new_text)    
        emb = embed([new_text])
        sentence_emb = tf.reshape(emb, [-1]).numpy()
        X_train_vectors[i].append(sentence_emb)

    X_train_vectors = list(zip(X_train_vectors[0], X_train_vectors[1], X_train_vectors[2], X_train_vectors[3]))
    for i in range(len(X_train_vectors)):
      X_train_vectors[i] = [item for sublist in X_train_vectors[i] for item in sublist]
    X_train_vectors = np.array(X_train_vectors)

     
    #creating testing vectors
    X_test_vectors = [[] for _ in range(len(X_test.columns))]
    for i in range(len(X_test.columns)): 
      for sentence in X_test[feature_columns[i]]:
        wordsList = nltk.word_tokenize(sentence) 
        text_tagged = nltk.pos_tag(wordsList) 
        new_text = []
        for word in text_tagged:
          new_text.append(word[0] + "/" + word[1])
        new_text = ' '.join(new_text)    
        emb = embed([new_text])
        sentence_emb = tf.reshape(emb, [-1]).numpy()
        X_test_vectors[i].append(sentence_emb)

    X_test_vectors = list(zip(X_test_vectors[0], X_test_vectors[1], X_test_vectors[2], X_test_vectors[3]))
    for i in range(len(X_test_vectors)):
      X_test_vectors[i] = [item for sublist in X_test_vectors[i] for item in sublist]
    X_test_vectors = np.array(X_test_vectors)

  if features == 'additional':
    X_train_vectors = [[] for _ in range(len(X_train.columns))]
    sentiment_score_response = []

    for i in range(len(X_train.columns)): 
      for sentence in X_train[feature_columns[i]]:
        sentence_ws = disambiguate(sentence)
        wordsList = nltk.word_tokenize(sentence) 
        text_tagged = nltk.pos_tag(wordsList) 
        
        new_text = []
        new_text2 = []
        idx = 0
        for word in text_tagged:
          new_text.append(word[0] + "/" + word[1] + "/" + (sentence_ws[idx][1].name() if (sentence_ws[idx] and sentence_ws[idx][1]) else ''))
          idx += 1

        new_text = ' '.join(new_text)
        
        emb = embed([new_text])
        sentence_emb = tf.reshape(emb, [-1]).numpy()
        X_train_vectors[i].append(sentence_emb)
        
        if i == 3:
          dic = sentiment_analyzer_scores(sentence)
          sentiment_score_response.append([dic['neg'], dic['neu'], dic['pos'], dic['compound']])


    X_train_vectors = list(zip(X_train_vectors[0], X_train_vectors[1], X_train_vectors[2], X_train_vectors[3]))
    
    for i in range(len(X_train_vectors)):
      X_train_vectors[i] = [item for sublist in X_train_vectors[i] for item in sublist]
    
    for i in range(len(X_train_vectors)):
      X_train_vectors[i] += sentiment_score_response[i]
    
    X_train_vectors = np.array(X_train_vectors)


    #testing data set
    X_test_vectors = [[] for _ in range(len(X_test.columns))]
    sentiment_score_response = []

    for i in range(len(X_test.columns)): 
      for sentence in X_test[feature_columns[i]]:
        sentence_ws = disambiguate(sentence)
        wordsList = nltk.word_tokenize(sentence) 
        text_tagged = nltk.pos_tag(wordsList) 
        
        new_text = []
        new_text2 = []
        idx = 0
        for word in text_tagged:
          new_text.append(word[0] + "/" + word[1] + "/" + (sentence_ws[idx][1].name() if (sentence_ws[idx] and sentence_ws[idx][1]) else ''))
          idx += 1

        new_text = ' '.join(new_text)
        
        emb = embed([new_text])
        sentence_emb = tf.reshape(emb, [-1]).numpy()
        X_test_vectors[i].append(sentence_emb)
        
        if i == 3:
          dic = sentiment_analyzer_scores(sentence)
          sentiment_score_response.append([dic['neg'], dic['neu'], dic['pos'], dic['compound']])

    X_test_vectors = list(zip(X_test_vectors[0], X_test_vectors[1], X_test_vectors[2], X_test_vectors[3]))
    for i in range(len(X_test_vectors)):
      X_test_vectors[i] = [item for sublist in X_test_vectors[i] for item in sublist]
    
    for i in range(len(X_test_vectors)):
      X_test_vectors[i] += sentiment_score_response[i]
    X_test_vectors = np.array(X_test_vectors)

  return ((X_train_vectors, y_train), (X_test_vectors, y_test))

In [ ]:
def train_model(X_train, y_train, classifier='SVC', plot_training=False):
  if classifier == 'SVC':
    #SVC Grid Search
    '''
    param_grid_SVC = {'C': [0.1, 1, 10, 100],  
                  'gamma': [0.0001, 0.001, 0.01, 0.1, 1, 10],
                  'kernel': ['rbf', 'linear']}  

    clf_SVC = GridSearchCV(SVC(), param_grid_SVC)
    clf_SVC.fit(X_train_vectors, y_train)
    print("Best parameter: {}, Best score: {}".format(clf_SVC.best_params_, clf_SVC.best_score_))
    '''

    model = SVC(C=10, kernel='rbf', gamma=1)
    model.fit(X_train, y_train)
    
  elif classifier == 'NN':
    model = keras.Sequential()
    model.add(keras.layers.Dense(units=128, input_shape=(X_train.shape[1],),
                                activation='relu'))
    model.add(keras.layers.Dropout(rate=0.5))
    model.add(keras.layers.Dense(units=64, activation='relu'))
    model.add(keras.layers.Dropout(rate=0.5))
    model.add(keras.layers.Dense(4, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(0.01),
                  metrics=['accuracy'])
    
    history = model.fit(
    X_train, y_train, epochs=9, batch_size=32,
    verbose=0, validation_split=0.1, shuffle=True)

    if plot_training:
      plt.plot(history.history['loss'], label='training data')
      plt.plot(history.history['val_loss'], label='validation data')
      plt.legend(loc="upper right")
      plt.show()

      plt.plot(history.history['accuracy'], label='training data')
      plt.plot(history.history['val_accuracy'], label='validation data')
      plt.legend(loc="upper left")
      plt.show()

  return model

In [ ]:
def test_model(model, X_test, y_test, classifier='SVC'):
  if classifier == 'SVC':
    pred_classes = model.predict(X_test)
  elif classifier == 'NN':
    pred_classes = np.argmax(model.predict(X_test), axis=-1)
  accuracy = accuracy_score(y_test, pred_classes)
  precision = precision_score(y_test, pred_classes, average='weighted')
  recall = recall_score(y_test, pred_classes, average='weighted')
  f1 = f1_score(y_test, pred_classes, average='weighted')
  print(f"Accuracy: {accuracy}")
  print('Precision: %f' %precision)
  print('Recall: %f' %recall)
  print('F1 score: %f' %f1)



Main

In [ ]:
df, df_test = load_data()
df, df_test = clean_data(df, df_test)

Baseline features

In [ ]:
(X_train, y_train), (X_test, y_test) = create_train_test_set(df, df_test, features='baseline')

In [ ]:
model_svc = train_model(X_train, y_train, classifier='SVC')

In [ ]:
test_model(model_svc, X_test, y_test, classifier='SVC')

In [ ]:
#model_nn = train_model(X_train, y_train, classifier='NN')

In [ ]:
#test_model(model_nn, X_test, y_test, classifier='NN')

Baseline + Additional features

In [ ]:
(X_train, y_train), (X_test, y_test) = create_train_test_set(df, df_test, features='additional')

In [ ]:
model_svc = train_model(X_train, y_train, classifier='SVC')

In [ ]:
test_model(model_svc, X_test, y_test, classifier='SVC')